In [1]:
# SNP scanner and pickler

# STATUS: WIP
# Run-time: unknown

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7497, clientId=10) # rkv tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=10) # rkv IBG live

In [2]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime
import os

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

# sd multiple for band
sigma = 2    # 2 sigma is about 95% probability
premium = 1.8    # e.g. 1.8 is 80% premium above

minHzn = 20   # minimum option horizon
maxHzn = 90   # maximum option horizon

base = 0.01   # Upper or Lower base multiple for prices

# market
exchange = 'SMART'
currency = 'USD'

# ... build the snp list

sym_chg_dict = {'BRK.B': 'BRK B', 'BRK/B': 'BRK B'} # Remap symbols in line with IBKR

snpurl = 'https://en.wikipedia.org/wiki/S%26P_100'
df_snp = pd.read_html(snpurl, header=0)[2]

df_snp.Symbol = df_snp.Symbol.map(sym_chg_dict).fillna(df_snp.Symbol)
df_snp['Type'] = 'Stock'

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

# read from row no 11, dropna and reset index
df_cboe = pd.read_excel(dls, header=11, 
                        usecols=[0,2,3]).loc[11:, :]\
                        .dropna(axis=0)\
                        .reset_index(drop=True)

# remove column names white-spaces and remap to IBKR
df_cboe.columns = df_cboe.columns.str.replace(' ', '')
df_cboe.Ticker = df_cboe.Ticker.map(sym_chg_dict).fillna(df_cboe.Ticker)

# list the equities
equities = [e for e in list(df_snp.Symbol) if e in list(df_cboe.Ticker)]

# filter and list the etfs
df_etf = df_cboe[df_cboe.ProductType == 'ETF'].reset_index(drop=True)
etfs = list(df_etf.Ticker)

# list the indexes
indexes = 'OEX,XEO,XSP,DJX'.split(',')

# Build a list of contracts
ss = [Stock(symbol=s, currency=currency, exchange=exchange) for s in set(equities+etfs)]
ixs = [Index(symbol=s,currency=currency, exchange='CBOE') for s in set(indexes)]

qcs = ib.qualifyContracts(*ss) # qualified underlyings

def df_pkl(contract):
    '''pickles df for contracts. Logic based on 1Scrip progra
    Arg: (contract) as object of either Stock or Index
    Returns: None'''

    # Get the symbol
    symbol = contract.symbol

    #... Get volatility, hi52 and lo52
    duration = '12 M'
    size = '1 day'
    bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                         durationStr=duration, barSizeSetting=size, 
                         whatToShow='TRADES', useRTH=True, 
                         formatDate=1, keepUpToDate=True)

    stDev = np.std(a=[b.close for b in bars], ddof=0)

    hi52 = max([b.high for b in bars])
    lo52 = min([b.low for b in bars])

    meanPrice = np.mean([b.close for b in bars])

    # Get the underlying's price
    ut = ib.reqTickers(contract)
    # ib.sleep(0.01)
    if  np.isnan(next(p.last for p in ut)):
        undPrice = next(p.close for p in ut)
    else:
        undPrice = next(p.last for p in ut)

    #... Get the option chains
    chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                          futFopExchange='', 
                          underlyingConId=contract.conId, underlyingSecType=contract.secType)

    chain = next(c for c in chains if c.exchange==exchange)

    strikes = sorted([s for s in chain.strikes 
                      if ((s < undPrice - stDev*sigma) | (s > undPrice + stDev*sigma))])

    expirations = sorted(exp for exp in chain.expirations 
                         if minHzn < (util.parseIBDatetime(exp)-datetime.datetime.now().date()).days < maxHzn)

    rights = ['P', 'C']

    #... Build and qualify the contracts
    contracts = [Option(symbol, expiration, strike, right, exchange)
                 for right in rights
                 for expiration in expirations
                 for strike in strikes]

    # qual_contracts = [c for c in contracts if c.conId]
    qual_contracts = [ib.qualifyContracts(*contracts[i:i+50]) for i in range(0, len(contracts), 50)]

    qc = [i for c in qual_contracts for i in c] # to remove empty []

    # remove unnecessary Puts and Calls
    tgts = [t for t in qc 
           if (t.strike < undPrice - stDev*sigma) & (t.right == 'P') | 
              (t.strike > undPrice + stDev*sigma) & (t.right == 'C')]

    tickers = [t for i in range(0, len(tgts), 20) for t in ib.reqTickers(*tgts[i:i + 20])]  # Get the tickers
    # ib.sleep(2)
    tickers = [t for i in range(0, len(tgts), 20) for t in ib.reqTickers(*tgts[i:i + 20])]  # Get the tickers

    lib_t = {t: catch(lambda: t.modelGreeks.undPrice) for t in tickers}
    und_t = [k for k, v in lib_t.items() if v is not None]

    # ... Build the dataframe
    ts = [(t.contract.conId, t.contract.symbol, t.contract.lastTradeDateOrContractMonth, t.contract.strike, t.contract.right, float(t.contract.multiplier), 
           t.modelGreeks.undPrice, t.contract.localSymbol, t.bid, t.bidSize, t.ask, t.askSize, t.close, t.modelGreeks.impliedVol, t.modelGreeks.delta, 
           t.modelGreeks.optPrice, t.modelGreeks.pvDividend, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta, hi52, lo52, meanPrice, stDev, t) 
          for t in und_t]

    cols = ['conId', 'ibSymbol', 'expiry', 'strike', 'right', 'lot', 'undPrice', 
            'localSymbol', 'bid', 'bidSize', 'ask', 'askSize', 'close', 'impliedVol', 'delta', 'optPrice', 
            'pvDividend', 'gamma', 'vega', 'theta', 'undHi', 'undLo', 'undMean', 'stdev', 'ticker']
    df1 = pd.DataFrame(ts, columns=cols).sort_values(by=['expiry', 'strike'], ascending=False).reset_index(drop=True)

    # get the margin
    order = Order(action='SELL', totalQuantity=1, orderType='MKT')
    whatif = [ib.whatIfOrder(contract, order) for contract in [t.contract for t in und_t]]
    margin = pd.to_numeric([w.initMarginChange for w in whatif])

    df1['whatif'] = whatif
    df1 ['margin'] = margin

    df1['expPrice'] = round(df1[['bid', 'ask', 'close']].max(axis=1)*premium/base)*base

    df1['dte'] = (pd.to_datetime(df1.expiry) - datetime.datetime.now()).dt.days

    # make negative dtes to 1 to accommodate last day option expiries
    df1.loc[df1.dte <= 0, 'dte'] = 1

    # calculate the rom
    df1['rom'] = (df1.expPrice*df1.lot)/df1.margin*252/df1.dte

    # remove calls with strike prices below undPrice and puts with strike prices above undPrice if any
    # and sort it by expiry and strike
    df = df1[~(((df1.strike < df1.undPrice) & (df1.right == 'C')) | ((df1.strike > df1.undPrice) & (df1.right == 'P')))].\
    sort_values(by=['expiry', 'strike'], ascending=[True, False]).reset_index(drop=True)

    # Pickle the dataframe if it is not empty
    if not df.empty:
        df.to_pickle('./zdata/pkls/'+symbol+'.pkl')
        

Started to throttle requests
Stopped to throttle requests


Wall time: 6.14 s


In [ ]:
%%time
# pickles the symbols in blocks of 50
[df_pkl(t) for i in range(0, len(qcs), 50) for t in qcs[i: i+50]]
ib.disconnect()

In [3]:
%%time
# Program to start from where connection failed.
fspath = './zdata/pkls'
fs = os.listdir(fspath)

# Take only pickle files. Remove directories
fs = [f for f in fs if f[-3:] == 'pkl']

Wall time: 1 ms


In [ ]:
# Get modified time, fail time and identify where the scrip has failed
fsmod = {f: os.path.getmtime(fspath + '/' + f) for f in fs}

failtime = max([v for k, v in fsmod.items()])
failscrip = [k[:-4] for k, v in fsmod.items() if v == failtime][0]
restartfrom = [q.symbol for q in qcs].index(failscrip)

# Get the remaining pickles
[df_pkl(t) for t in qcs[restartfrom + 1:]]
ib.disconnect()

In [ ]:
# logic to get remaining pickles in total (not only from failed time)
completed = [s[:-4] for s in fs]
allsymbols = [q.symbol for q in qcs]
remainingsyms = [a for a in allsymbols if a not in completed]
remainingqcs = [q for q in qcs if q.symbol not in remainingsyms]

[df_pkl(q) for q in remainingqcs]
ib.disconnect()

Error 200, reqId 15898: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=12.0, right='P', exchange='SMART')
Error 200, reqId 15899: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=13.0, right='P', exchange='SMART')
Error 200, reqId 15900: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=14.0, right='P', exchange='SMART')
Error 200, reqId 15902: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=16.0, right='P', exchange='SMART')
Error 200, reqId 15901: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=15.0, right='P', exchange='SMART')
Error 200, reqId 15903: No security

Error 200, reqId 15967: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190201', strike=41.5, right='P', exchange='SMART')
Error 200, reqId 15966: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190201', strike=41.0, right='P', exchange='SMART')
Error 200, reqId 15970: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190201', strike=43.0, right='P', exchange='SMART')
Error 200, reqId 15969: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190201', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 15971: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190201', strike=43.5, right='P', exchange='SMART')
Error 200, reqId 15972: No security

Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=14.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=15.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=16.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=17.0, right='P', exchange='SMART')
Error 200, reqId 16001: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=36.0, right='P', exchange='SMART')
Error 200, reqId 16003: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=37.0, right='P', exchange='SMART')
Error 200, reqId 16002: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrC

Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=37.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=37.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=38.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=38.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=39.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=39.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=40.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190208', strike=40.5, right='P', exchange='SMART')
Unknown contract: Option

Error 200, reqId 16077: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190215', strike=59.0, right='P', exchange='SMART')
Error 200, reqId 16078: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=12.0, right='C', exchange='SMART')
Error 200, reqId 16079: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=13.0, right='C', exchange='SMART')
Error 200, reqId 16080: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=14.0, right='C', exchange='SMART')
Error 200, reqId 16081: No security definition has been found for the request, contract: Option(symbol='UNG', lastTradeDateOrContractMonth='20190125', strike=15.0, right='C', exchange='SMART')
Error 200, reqId 16082: No security

...

Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=1975.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=1985.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=1995.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2005.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2010.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2015.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2025.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2030.0, right='C', exchange='SMART')


Error 200, reqId 33712: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2315.0, right='C', exchange='SMART')
Error 200, reqId 33713: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2320.0, right='C', exchange='SMART')
Error 200, reqId 33714: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2325.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2085.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2095.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2105.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN

Error 200, reqId 33731: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2410.0, right='C', exchange='SMART')
Error 200, reqId 33732: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2415.0, right='C', exchange='SMART')
Error 200, reqId 33733: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2420.0, right='C', exchange='SMART')
Error 200, reqId 33736: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2435.0, right='C', exchange='SMART')
Error 200, reqId 33734: No security definition has been found for the request, contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2425.0, right='C', exchange='SMART')
Error 200, reqId 337

Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2470.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2475.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2480.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2485.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2490.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2495.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2500.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AMZN', lastTradeDateOrContractMonth='20190315', strike=2600.0, right='C', exchange='SMART')


Error 10197, reqId 33855: No market data during competing live session, contract: Option(conId=348034837, symbol='AMZN', lastTradeDateOrContractMonth='20190208', strike=1040.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMZN  190208P01040000', tradingClass='AMZN')
Error 10197, reqId 33918: No market data during competing live session, contract: Option(conId=315620441, symbol='AMZN', lastTradeDateOrContractMonth='20190215', strike=1000.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMZN  190215P01000000', tradingClass='AMZN')
Error 10197, reqId 33916: No market data during competing live session, contract: Option(conId=342776463, symbol='AMZN', lastTradeDateOrContractMonth='20190215', strike=990.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMZN  190215P00990000', tradingClass='AMZN')
Error 10197, reqId 33920: No market data during competing live session, contract: Option(conId=315620448, s